<a href="https://colab.research.google.com/github/tomheston/fragility-metrics/blob/main/notebooks/continuous_outcomes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title
# Fragility Metrics Toolkit: Continuous Outcomes (Welch t-test)
# 20-NOV-2025
# Fully aligned with FRAGILITY_METRICS.md v9.5 §3.7 CFQ/CFS and §4 MeCI
#
# Input: m1, sd1, n1  (group 1), m2, sd2, n2  (group 2)
# Output: p (Welch t p-value), fr (CFS/CFQ), nb (MeCI)
#
# Formulas (exact v9.5):
# - CFS  = ||T| – t*|   → positive distance to the α=0.05 boundary (whichever side we are on)
# - CFQ  = CFS / (1 + CFS)                  → fragility quotient 0–1
# - MeCI = D / (1 + D)   where D = |m1-m2| / SE_diff   → robustness 0–1
#
# Note: This calculator requires the standard deviation (SD) of each group, not the standard error of the mean (SE).
# Most papers plot or report mean ± SE (because error bars look smaller).
# If only SE is given, convert using: SD = SE × √n   (n = sample size of the group)
#
# IF YOU USE THIS CALCULATOR PLEASE CITE:
# Heston, T. F. (2025). Fragility Metrics Toolkit [Software]. Zenodo. https://doi.org/10.5281/zenodo.17254763
#
# © Thomas F. Heston 2025. CC-BY 4.0

import numpy as np
from scipy.stats import t

ALPHA = 0.05

# ---------- Core calculations ----------
def welch_stats(m1, sd1, n1, m2, sd2, n2):
    se1 = sd1 / np.sqrt(n1)
    se2 = sd2 / np.sqrt(n2)
    se_diff = np.sqrt(se1**2 + se2**2)
    t_stat = (m1 - m2) / se_diff
    df = (se1**2 + se2**2)**2 / (se1**4 / (n1 - 1) + se2**4 / (n2 - 1))
    p_val = 2 * t.sf(np.abs(t_stat), df)
    t_crit = t.ppf(1 - ALPHA/2, df)

    # Correct CFS: always the positive distance to the boundary (whichever side we are on)
    cfs = abs(abs(t_stat) - t_crit)
    cfq = cfs / (1 + cfs)
    meci = abs(m1 - m2) / se_diff / (1 + abs(m1 - m2) / se_diff)

    return t_stat, p_val, se_diff, df, cfs, cfq, meci

# ---------- High-level ----------
def continuous_outcomes(m1, sd1, n1, m2, sd2, n2):
    if n1 < 2 or n2 < 2:
        raise ValueError("Sample sizes must be ≥2")
    if sd1 <= 0 or sd2 <= 0:
        raise ValueError("Standard deviations must be positive")

    t_stat, p_val, se_diff, df, cfs, cfq, meci = welch_stats(m1, sd1, n1, m2, sd2, n2)

    return {
        "p": p_val,
        "fr": {"CFS": cfs, "CFQ": cfq},
        "nb": {"MeCI": meci}
    }

# ---------- Narrative ----------
def generate_narrative(res, m1, sd1, n1, m2, sd2, n2):
    diff = abs(m1 - m2)
    se_diff = (sd1**2/n1 + sd2**2/n2)**0.5

    lines = [
        f"Group 1: mean = {m1:.4f}, sd = {sd1:.4f}, n = {n1}",
        f"Group 2: mean = {m2:.4f}, sd = {sd2:.4f}, n = {n2}",
        f"Mean difference = {diff:.4f}  (SE = {se_diff:.4f})",
        "",
        "p: p-value = {:.6f} ({})".format(res['p'], "significant" if res['p'] <= ALPHA else "not significant"),
        "fr: CFQ = {:.6f} ({})".format(
            res['fr']['CFQ'],
            "extremely fragile" if res['fr']['CFQ'] < 0.01 else
            "very fragile" if res['fr']['CFQ'] < 0.05 else
            "fragile" if res['fr']['CFQ'] < 0.10 else
            "moderately stable" if res['fr']['CFQ'] < 0.25 else
            "very stable"
        ),
        "nb: MeCI = {:.6f} → {}".format(
            res['nb']['MeCI'],
            "at neutrality boundary" if res['nb']['MeCI'] < 0.05 else
            "near neutrality" if res['nb']['MeCI'] < 0.10 else
            "moderately separated" if res['nb']['MeCI'] < 0.25 else
            "clearly separated" if res['nb']['MeCI'] < 0.50 else
            "far from neutrality"
        )
    ]

    if res['p'] <= ALPHA:
        # Claim: difference exists
        if res['fr']['CFQ'] > 0.25 and res['nb']['MeCI'] > 0.5:
            summary = "Claim of difference is supported, very stable, and far from neutrality."
        else:
            summary = "Claim of difference is supported but fragile and/or close to neutrality."
    else:
        # Claim: no difference
        if res['fr']['CFQ'] > 0.25:
            summary = "Claim of no difference is supported and very stable."
        else:
            summary = "Claim of no difference is supported but fragile (close to significance)."

    lines.append(f"\n{summary}")

    return "\n".join(lines)

# ---------- CLI ----------
def main():
    print("Continuous Outcomes Calculator (Welch t) – v9.5 compliant\n")
    print("Note: this calculator requires input of standard deviations (SD) and not standard error (SE).")
    print("To convert from SE to SD use SD = SE x √n\n")
    print("Group 1")
    m1 = float(input("Mean: "))
    sd1 = float(input("SD: "))
    n1 = int(input("n: "))
    print("\nGroup 2")
    m2 = float(input("Mean: "))
    sd2 = float(input("SD: "))
    n2 = int(input("n: "))

    res = continuous_outcomes(m1, sd1, n1, m2, sd2, n2)

    print("\n================ p–fr–nb ================")
    print(f"p: p-value = {res['p']:.6f} ({'significant' if res['p'] <= ALPHA else 'not significant'})")
    print(f"fr: CFQ = {res['fr']['CFQ']:.6f}")
    print(f"nb: MeCI = {res['nb']['MeCI']:.6f}")
    print("=========================================\n")

    print("Interpretation:")
    print(generate_narrative(res, m1, sd1, n1, m2, sd2, n2))

if __name__ == "__main__":
    main()

Continuous Outcomes Calculator (Welch t) – v9.5 compliant

Note: this calculator requires input of standard deviations (SD) and not standard error (SE).
To convert from SE to SD use SD = SE x √n

Group 1
Mean: 33
SD: 5
n: 55

Group 2
Mean: 34
SD: 6
n: 55

================ p–fr–nb ================
p: p-value = 0.344531 (not significant)
fr: CFQ = 0.508202
nb: MeCI = 0.487060

Interpretation:
Group 1: mean = 33.0000, sd = 5.0000, n = 55
Group 2: mean = 34.0000, sd = 6.0000, n = 55
Mean difference = 1.0000  (SE = 1.0531)

p: p-value = 0.344531 (not significant)
fr: CFQ = 0.508202 (very stable)
nb: MeCI = 0.487060 → clearly separated

Claim of no difference is supported and very stable.
